In [32]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
train=pd.read_csv('train.tsv',delimiter='\t')
test=pd.read_csv('test.tsv',delimiter='\t')
df=train.append(test).reset_index(drop=True)
df['horsepower']=df['horsepower'].replace('?',np.nan)
df['horsepower']=df['horsepower'].astype(np.float64)
df['horsepower']=df['horsepower'].fillna(df['horsepower'].median())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [40]:
df.tail()

,acceleration,car name,cylinders,displacement,horsepower,id,model year,mpg,origin,weight
393,14.0,audi 100ls,4,114.0,91.0,391,73,NaN,2,2582.0
394,14.4,dodge colt,4,156.0,105.0,392,80,NaN,1,2800.0
395,14.8,buick opel isuzu deluxe,4,111.0,80.0,393,77,NaN,1,2155.0
396,11.1,pontiac grand prix lj,8,400.0,180.0,394,77,NaN,1,4220.0
397,14.5,volkswagen rabbit custom,4,97.0,78.0,397,77,NaN,2,1940.0


In [41]:
feats_for_poly=[f for f in df.columns if f not in ['id','mpg','car name']]
df_poly=df[feats_for_poly]

In [42]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(interaction_only=True,include_bias=False)
df_poly=poly.fit_transform(df_poly)

In [43]:
poly_columns=poly.get_feature_names(feats_for_poly)

In [44]:
df_poly=pd.DataFrame(df_poly,columns=poly_columns)

In [45]:
df_car=pd.get_dummies(df['car name'])

In [46]:
df=pd.concat([df_poly,df_car,df[['id','mpg']]],axis=1)

In [47]:
df.shape

(398, 335)

In [48]:
import lightgbm as lgb
train_df=df[df['mpg'].notnull()]
test_df=df[df['mpg'].isnull()]

feats=[f for f in train_df.columns if f not in ['index','id','mpg']]
train_X=train_df[feats]
train_y=train_df['mpg']

train_X,train_y=train_df[feats],train_df['mpg']
test_X=test_df[feats]
gbm_reg=lgb.LGBMRegressor(objective='regression',
                     num_leaves=31,
                     n_estimators=100)
gbm_reg.fit(train_X,train_y,
           verbose=0)
pred_sub=gbm_reg.predict(test_X)

In [49]:
sub=pd.DataFrame(test['id'])
sub['mpg']=pred_sub

In [50]:
sub.to_csv('0909_poly.csv',header=False,index=False)

In [52]:
df['horsepower/weight']=df['horsepower']/df['weight']

In [55]:
import lightgbm as lgb
train_df=df[df['mpg'].notnull()]
test_df=df[df['mpg'].isnull()]

feats=[f for f in train_df.columns if f not in ['index','id','mpg']]
train_X=train_df[feats]
train_y=train_df['mpg']

train_X,train_y=train_df[feats],train_df['mpg']
test_X=test_df[feats]
gbm_reg=lgb.LGBMRegressor(objective='regression',
                     num_leaves=31,
                     n_estimators=100)
gbm_reg.fit(train_X,train_y,
           verbose=0)
pred_sub=gbm_reg.predict(test_X)
sub=pd.DataFrame(test['id'])
sub['mpg']=pred_sub
sub.to_csv('0909_poly.csv',header=False,index=False)